In [2]:
import json
import satsearch
import geopandas as gpd
import numpy as np
import pandas as pd
import warnings
from shapely.geometry import  Polygon
import rioxarray
import satstac
import os
warnings.filterwarnings("ignore")

In [16]:
import data.sentinel_cog as sc


In [17]:
#data from http://geojson.io 
data={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              38.54827880859375,
              36.90378362619561
            ],
            [
              40.0067138671875,
              36.90378362619561
            ],
            [
              40.0067138671875,
              37.89219554724437
            ],
            [
              38.54827880859375,
              37.89219554724437
            ],
            [
              38.54827880859375,
              36.90378362619561
            ]
          ]
        ]
      }
    }
  ]
}

In [18]:
adiyaman={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              38.20838928222656,
              37.47976234695507
            ],
            [
              38.51600646972656,
              37.47976234695507
            ],
            [
              38.51600646972656,
              37.68083151896963
            ],
            [
              38.20838928222656,
              37.68083151896963
            ],
            [
              38.20838928222656,
              37.47976234695507
            ]
          ]
        ]
      }
    }
  ]
}

In [19]:
datajson=json.dumps(data)
target_area=gpd.read_file(datajson)


In [20]:
tiles_intersection,tile_map=sc.find_sentinel_tile(target_area)
tile_map

In [21]:
def show_result_map(result=None,items_list=[],target_area=None,overview=False):
    center = [38,35]
    zoom = 6
    if not items_list:
        items = result.items()
        m = folium.Map(location=center, zoom_start=zoom)
        for item in items:
            if overview:
                #get overview band
                #get overview band
                band_url=item.assets['thumbnail']['href'] 
                pol=Polygon(item.geometry['coordinates'][0])
                folium.raster_layers.ImageOverlay(
                    image=band_url,
                    name=item.properties['sentinel:product_id'],
                    bounds=[[min(pol.exterior.coords.xy[0]),min(pol.exterior.coords.xy[1])],[max(pol.exterior.coords.xy[0]),max(pol.exterior.coords.xy[1])] ],
                    #bounds=[item.geometry['coordinates'][0][0], item.geometry['coordinates'][0][3]],
                    opacity=1,
                    interactive=True,
                    cross_origin=True,
                    zindex=1,
                    #alt="Wikipedia File:Mercator projection SW.jpg",
                ).add_to(m)   

                folium.LayerControl().add_to(m)
            else:
                # Create a string for the name containing the path and row of this Polygon
                name = item.properties['sentinel:product_id']
                # Create the folium geometry of this Polygon 
                g = folium.GeoJson(item.geometry, name=name)
                # Add a folium Popup object with the name string
                g.add_child(folium.Popup(name))
                # Add the object to the map
                g.add_to(m)

        if target_area:
            m.add_child(folium.GeoJson(target_area.__geo_interface__, name='Area of Study', 
                           style_function=lambda x: {'color': 'red', 'alpha': 0}))
    
        folium.LayerControl().add_to(m)
        return m

      
    else:
        m = folium.Map(location=center, zoom_start=zoom)
        for item in items_list:
            if overview:
                #get overview band
                band_url=item.assets['thumbnail']['href'] 
                pol=Polygon(item.geometry['coordinates'][0])
                folium.raster_layers.ImageOverlay(
                    image=band_url,
                    name=item.properties['sentinel:product_id'],
                    bounds=[[min(pol.exterior.coords.xy[0]),min(pol.exterior.coords.xy[1])],[max(pol.exterior.coords.xy[0]),max(pol.exterior.coords.xy[1])] ],
                    #bounds=[item.geometry['coordinates'][0][0], item.geometry['coordinates'][0][3]],
                    opacity=1,
                    interactive=True,
                    cross_origin=True,
                    zindex=1,
                    #alt="Wikipedia File:Mercator projection SW.jpg",
                ).add_to(m)   

            else:
                # Create a string for the name containing the path and row of this Polygon
                name = item.properties['sentinel:product_id']
                # Create the folium geometry of this Polygon 
                g = folium.GeoJson(item.geometry, name=name)
                # Add a folium Popup object with the name string
                g.add_child(folium.Popup(name))
                # Add the object to the map
                g.add_to(m)

                

        if target_area:
            m.add_child(folium.GeoJson(target_area.__geo_interface__, name='Area of Study', 
                           style_function=lambda x: {'color': 'red', 'alpha': 0}))
    
        folium.LayerControl().add_to(m)
        return m

In [22]:
tiles_intersection

,geometry
0,"POLYGON ((38.54828 36.90378, 40.00671 36.90378..."


In [24]:
import data.sentinel_cog as sc

In [25]:
boundry=list(target_area.geometry.bounds.values[0]) #boundry from your AOI
bbox=[boundry[0],boundry[1],boundry[2],boundry[3]] #(min lon, min lat, max lon, max lat)
dates = '2018-07-01/2018-07-12'
band_list=['B02','B03','B04','B08']
cloud_percentage=20

In [26]:
stac_result=sc.find_stac_result(bbox,dates,cloud_percentage)

In [27]:
tile_list=sc.create_tiles_list(stac_result)
tile_list

['37SDA', '37SDB', '37SDC', '37SEA', '37SEB', '37SEC']

In [46]:
items=stac_result.items()

In [34]:
coverage=sorted(items.properties('sentinel:data_coverage'))

In [43]:
import pandas as pd

In [50]:
items_json=items.geojson()
items_json=json.dumps(items_json)
df=gpd.read_file(items_json)


In [51]:
df.head()

,id,datetime,platform,constellation,gsd,data_coverage,view:off_nadir,eo:cloud_cover,proj:epsg,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,created,updated,sentinel:valid_cloud_cover,sentinel:utm_zone,sentinel:data_coverage,geometry
0,S2A_37SDA_20180712_0_L2A,2018-07-12T08:15:37+00:00,sentinel-2a,sentinel-2,10,100.0,0,0.02,32637,S,DA,0,S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_2...,2020-09-06T11:24:04.973000+00:00,2020-09-06T11:24:04.973000+00:00,True,37,100.0,"POLYGON ((37.88933 36.05159, 37.87508 37.04124..."
1,S2A_37SEC_20180712_0_L2A,2018-07-12T08:15:37+00:00,sentinel-2a,sentinel-2,10,NaN,0,4.11,32637,S,EC,0,S2A_MSIL2A_20180712T080611_N0208_R078_T37SEC_2...,2020-08-31T01:16:46.973000+00:00,2020-08-31T01:16:46.973000+00:00,True,37,100.0,"POLYGON ((40.24785 37.85284, 38.99978 37.85945..."
2,S2A_37SDB_20180712_0_L2A,2018-07-12T08:15:37+00:00,sentinel-2a,sentinel-2,10,100.0,0,0.00,32637,S,DB,0,S2A_MSIL2A_20180712T080611_N0208_R078_T37SDB_2...,2020-09-26T15:35:33.570000+00:00,2020-09-26T15:35:33.570000+00:00,True,37,100.0,"POLYGON ((37.87638 36.95257, 37.86148 37.94207..."
3,S2A_37SEA_20180712_0_L2A,2018-07-12T08:15:37+00:00,sentinel-2a,sentinel-2,10,NaN,0,0.00,32637,S,EA,0,S2A_MSIL2A_20180712T080611_N0208_R078_T37SEA_2...,2020-08-31T01:12:31.679001+00:00,2020-08-31T01:12:31.679001+00:00,True,37,100.0,"POLYGON ((40.21879 36.05054, 38.99979 36.05673..."
4,S2A_37SEB_20180712_0_L2A,2018-07-12T08:15:37+00:00,sentinel-2a,sentinel-2,10,NaN,0,0.15,32637,S,EB,0,S2A_MSIL2A_20180712T080611_N0208_R078_T37SEB_2...,2020-08-31T04:00:21.618000+00:00,2020-08-31T04:00:21.618000+00:00,True,37,100.0,"POLYGON ((40.23299 36.95149, 38.99979 36.95789..."


In [41]:
try:
    if not items:
        raise IndexError
except:
    print(5555)

5555


In [32]:
items.filter('sentinel:data_coverage',[])

In [35]:
#drop_list=['37SDA', '37SDC', '37SEA', '37SEB', '37SEC']

In [36]:
#tiles_intersection=sc.drop_tile(tiles_intersection,drop_list)
#tiles_intersection

In [34]:
from dask.distributed import Client, LocalCluster
import multiprocessing as mp
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
import satsearch

     
def find_stac_result(target_aoi,date,max_cloud=10):
    URL='https://earth-search.aws.element84.com/v0'
    with LocalCluster(n_workers=int(0.5 * mp.cpu_count()),
                          processes=True,
                          threads_per_worker=2,
                          memory_limit='2GB',
                          #ip='tcp://localhost:9895',
                          ) as cluster, Client(cluster) as client:
        results = satsearch.Search.search(url=URL,
                                          collections=['sentinel-s2-l2a-cogs'], # note collection='sentinel-s2-l2a-cogs' doesn't work
                                          datetime=date,
                                          bbox=target_aoi,
                                          query={'eo:cloud_cover': {'lt':max_cloud}}, )
    return results

def create_tiles_list(stac_result):
    items = stac_result.items()
    items_json=items.geojson()
    items_json=json.dumps(items_json)
    df=gpd.read_file(items_json)
    df['tile']=df.apply(lambda row: str(row['sentinel:utm_zone'])+row['sentinel:latitude_band']+row['sentinel:grid_square'], axis=1)
    tiles_list = sorted(df['tile'].unique().tolist())
    return tiles_list
    

In [37]:
def find_sentinel_item(stac_result,tiles_list=[],min_coverage=95,max_cloud=2):
    if tiles_list:
        tile_result_list=[]
        with LocalCluster(n_workers=int(0.5 * mp.cpu_count()),
                          processes=True,
                          threads_per_worker=2,
                          memory_limit='2GB',
                          #ip='tcp://localhost:9895',
                          ) as cluster, Client(cluster) as client:
            
            for tile in tiles_list:
                #find best image for target tile 
                tile_number=int(tile[0:2])
                lat_band=tile[2]
                grid_sq=tile[3:]
                items = stac_result.items()
                items.filter('sentinel:utm_zone',[tile_number,])
                items.filter('sentinel:latitude_band',[lat_band])
                items.filter('sentinel:grid_square', [grid_sq])
                try:
                    coverage=sorted(items.properties('sentinel:data_coverage'))
                    coverage=[c for c in coverage if c>min_coverage]
                    items.filter('sentinel:data_coverage',coverage)
                    if 100 in items.properties('sentinel:data_coverage'):
                        #get the index of value that data_coverage==100
                        dc_index=[i for i, x in enumerate(items) if items[i].properties['sentinel:data_coverage']==100]
                        #get images cloud info
                        filtered_list=[items[x].properties['eo:cloud_cover'] for i, x in enumerate(dc_index) if items[x].properties['eo:cloud_cover']<max_cloud]
                        #get first cloud cover after sorting
                        try:
                            # if data exist we use filter method
                            selected_item=sorted(filtered_list)[0]
                            items.filter('sentinel:data_coverage',[100])
                        except:
                            selected_item=sorted(items.properties('eo:cloud_cover'))[0]
            
                        items.filter('eo:cloud_cover', [selected_item])
                        tile_result_list.append(items[0])
    
                    else:
                        selected_item=sorted(items.properties('eo:cloud_cover'))[0]
                        #select best image
                        items.filter('eo:cloud_cover', [selected_item])
                        #get newest image
                        tile_result_list.append(items[0])
                              
                except:
                    # threshold ekle -2 yerine
                    coverage=sorted(items.properties('sentinel:data_coverage'))[-2:]
                    items.filter('sentinel:data_coverage',coverage)
                    selected_item=sorted(items.properties('eo:cloud_cover'))[0]
                    #select best image
                    items.filter('eo:cloud_cover', [selected_item])
                    #get newest image
                    tile_result_list.append(items[0])
            
            return tile_result_list
    
    else:
        # code define best image for each tile from stac result
        # function return list of tile's information
        tiles_list=create_tiles_list(stac_result)
        #we collect each tile's result
        tile_result_list=[]
        with LocalCluster(n_workers=int(0.5 * mp.cpu_count()),
                          processes=True,
                          threads_per_worker=2,
                          memory_limit='2GB',
                          #ip='tcp://localhost:9895',
                          ) as cluster, Client(cluster) as client:
            for tile in tiles_list:
                tile_number=int(tile[0:2])
                lat_band=tile[2]
                grid_sq=tile[3:]
                items = stac_result.items()
                items.filter('sentinel:utm_zone',[tile_number,])
                items.filter('sentinel:latitude_band',[lat_band])
                items.filter('sentinel:grid_square', [grid_sq])
                try:
                    coverage=sorted(items.properties('sentinel:data_coverage'))
                    coverage=[c for c in coverage if c>min_coverage]
                    items.filter('sentinel:data_coverage',coverage)
                    if 100 in items.properties('sentinel:data_coverage'):
                        #get the index of value that data_coverage==100
                        dc_index=[i for i, x in enumerate(items) if items[i].properties['sentinel:data_coverage']==100]
                        #get images cloud info
                        filtered_list=[items[x].properties['eo:cloud_cover'] for i, x in enumerate(dc_index) if items[x].properties['eo:cloud_cover']<max_cloud]
                        #get first cloud cover after sorting
                        try:
                            # if data exist we use filter method
                            selected_item=sorted(filtered_list)[0]
                            items.filter('sentinel:data_coverage',[100])
    
                        except:
                            selected_item=sorted(items.properties('eo:cloud_cover'))[0]
            
                        items.filter('eo:cloud_cover', [selected_item])
                        tile_result_list.append(items[0])
                        
                    else:
                        selected_item=sorted(items.properties('eo:cloud_cover'))[0]
                        #select best image
                        items.filter('eo:cloud_cover', [selected_item])
                        #get newest image
                        tile_result_list.append(items[0])
                              
                except:
                    coverage=sorted(items.properties('sentinel:data_coverage'))[-2:]
                    items.filter('sentinel:data_coverage',coverage)
                    selected_item=sorted(items.properties('eo:cloud_cover'))[0]
                    #select best image
                    items.filter('eo:cloud_cover', [selected_item])
                    #get newest image
                    tile_result_list.append(items[0])
            
            return tile_result_list

In [40]:
%%time
stac_result=find_stac_result(bbox,dates,cloud_percentage)
items_list=find_sentinel_item(stac_result=stac_result)
#Wall time: 37.3 s

Task was destroyed but it is pending!
task: <Task pending coro=<HTTP1ServerConnection._server_request_loop() running at C:\Users\akif\.conda\envs\geo\lib\site-packages\tornado\http1connection.py:823> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x000002103EB6C438>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\akif\.conda\envs\geo\lib\site-packages\tornado\ioloop.py:688]>
Task was destroyed but it is pending!
task: <Task pending coro=<HTTP1ServerConnection._server_request_loop() running at C:\Users\akif\.conda\envs\geo\lib\site-packages\tornado\http1connection.py:823> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x000002103E4AA3D8>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at C:\Users\akif\.conda\envs\geo\lib\site-packages\tornado\ioloop.py:688]>
Task was destroyed but it is pending!
task: <Task pending coro=<HTTP1ServerConnection._server_request_loop() running at C:\Users\akif\.conda\envs\geo\lib\site-packages\tornado\http1connection.py

Wall time: 31.5 s


In [61]:
sc.show_result_map(items_list=items_list,overview=True,target_area=target_area)

In [62]:
sc.show_items_list(items_list[0:1])

[{'tile_name': '37SDA',
  'tile_images': {'image_name': 'S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148',
   'bands': {'B01': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/7/S2A_37SDA_20180712_0_L2A/B01.tif',
    'B02': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/7/S2A_37SDA_20180712_0_L2A/B02.tif',
    'B03': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/7/S2A_37SDA_20180712_0_L2A/B03.tif',
    'B04': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/7/S2A_37SDA_20180712_0_L2A/B04.tif',
    'B05': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/7/S2A_37SDA_20180712_0_L2A/B05.tif',
    'B06': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2018/7/S2A_37SDA_20180712_0_L2A/B06.tif',
    'B07': 'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-co

In [63]:
df=sc.show_result_df(result=stac_result)

In [64]:
df.head()

,id,datetime,platform,constellation,gsd,data_coverage,view:off_nadir,eo:cloud_cover,proj:epsg,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,created,updated,sentinel:valid_cloud_cover,sentinel:utm_zone,sentinel:data_coverage,geometry
0,S2A_37SDA_20180712_0_L2A,2018-07-12 08:15:37+00:00,sentinel-2a,sentinel-2,10,100.0,0,0.02,32637,S,DA,0,S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_2...,2020-09-06T11:24:04.973000+00:00,2020-09-06T11:24:04.973000+00:00,True,37,100.0,"POLYGON ((37.88933 36.05159, 37.87508 37.04124..."
1,S2A_37SEC_20180712_0_L2A,2018-07-12 08:15:37+00:00,sentinel-2a,sentinel-2,10,NaN,0,4.11,32637,S,EC,0,S2A_MSIL2A_20180712T080611_N0208_R078_T37SEC_2...,2020-08-31T01:16:46.973000+00:00,2020-08-31T01:16:46.973000+00:00,True,37,100.0,"POLYGON ((40.24785 37.85284, 38.99978 37.85945..."
2,S2A_37SDB_20180712_0_L2A,2018-07-12 08:15:37+00:00,sentinel-2a,sentinel-2,10,100.0,0,0.00,32637,S,DB,0,S2A_MSIL2A_20180712T080611_N0208_R078_T37SDB_2...,2020-09-26T15:35:33.570000+00:00,2020-09-26T15:35:33.570000+00:00,True,37,100.0,"POLYGON ((37.87638 36.95257, 37.86148 37.94207..."
3,S2A_37SEA_20180712_0_L2A,2018-07-12 08:15:37+00:00,sentinel-2a,sentinel-2,10,NaN,0,0.00,32637,S,EA,0,S2A_MSIL2A_20180712T080611_N0208_R078_T37SEA_2...,2020-08-31T01:12:31.679001+00:00,2020-08-31T01:12:31.679001+00:00,True,37,100.0,"POLYGON ((40.21879 36.05054, 38.99979 36.05673..."
4,S2A_37SEB_20180712_0_L2A,2018-07-12 08:15:37+00:00,sentinel-2a,sentinel-2,10,NaN,0,0.15,32637,S,EB,0,S2A_MSIL2A_20180712T080611_N0208_R078_T37SEB_2...,2020-08-31T04:00:21.618000+00:00,2020-08-31T04:00:21.618000+00:00,True,37,100.0,"POLYGON ((40.23299 36.95149, 38.99979 36.95789..."


In [24]:
df[df['sentinel:grid_square']=='DC']

,id,datetime,platform,constellation,gsd,data_coverage,view:off_nadir,eo:cloud_cover,proj:epsg,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,created,updated,sentinel:valid_cloud_cover,sentinel:utm_zone,sentinel:data_coverage,geometry
4,S2A_37SDC_20200711_0_L2A,2020-07-11 08:19:31+00:00,sentinel-2a,sentinel-2,10,NaN,0,0.44,32637,S,DC,0,S2A_MSIL2A_20200711T080611_N0214_R078_T37SDC_2...,2020-08-18T11:31:52.827999+00:00,2020-08-18T11:31:52.827999+00:00,True,37,99.63,"POLYGON ((37.86284 37.85396, 37.85001 38.67226..."
7,S2B_37SDC_20200709_0_L2A,2020-07-09 08:29:24+00:00,sentinel-2b,sentinel-2,10,NaN,0,0.00,32637,S,DC,0,S2B_MSIL2A_20200709T081609_N0214_R121_T37SDC_2...,2020-08-18T12:42:44.929001+00:00,2020-08-18T12:42:44.929001+00:00,True,37,67.75,"POLYGON ((37.86284 37.85396, 37.84726 38.84330..."
14,S2B_37SDC_20200706_0_L2A,2020-07-06 08:19:29+00:00,sentinel-2b,sentinel-2,10,NaN,0,6.60,32637,S,DC,0,S2B_MSIL2A_20200706T080609_N0214_R078_T37SDC_2...,2020-08-18T11:37:05.921000+00:00,2020-08-18T11:37:05.921000+00:00,True,37,99.56,"POLYGON ((37.86284 37.85396, 37.85028 38.65560..."
22,S2A_37SDC_20200701_0_L2A,2020-07-01 08:19:32+00:00,sentinel-2a,sentinel-2,10,NaN,0,5.48,32637,S,DC,0,S2A_MSIL2A_20200701T080611_N0214_R078_T37SDC_2...,2020-08-18T11:29:55.672001+00:00,2020-08-18T11:29:55.672001+00:00,True,37,99.69,"POLYGON ((37.86284 37.85396, 37.84983 38.68373..."


In [43]:
df[df['sentinel:grid_square']=='DB'].iloc[1]['sentinel:product_id']

'S2B_MSIL2A_20180707T080609_N0208_R078_T37SDB_20180707T111717'

In [44]:
items_list

[S2A_37SDA_20180712_0_L2A,
 S2A_37SDB_20180712_0_L2A,
 S2B_37SDC_20180707_0_L2A,
 S2A_37SEA_20180712_0_L2A,
 S2A_37SEB_20180702_0_L2A,
 S2B_37SEC_20180707_0_L2A]

In [26]:
sc.download_image(item_list=items_list,band_list=['thumbnail'])

'./sentinel_cog'

In [77]:
%%time
subset=sc.download_subset_image(download_status=False,item_list=items_list,aoi=data,target_epsg='',
                        band_list=['B02','B08'],download_path='./sentinel_cog',name_suffix='',auto_folder=True)

Wall time: 17.4 s


In [79]:
img1=subset[0]
img1

{'image_name': 'S2A_MSIL2A_20180712T080611_N0208_R078_T37SDA_20180712T105148',
 'B02': <xarray.DataArray (band: 1, y: 1575, x: 5000)>
 dask.array<getitem, shape=(1, 1575, 5000), dtype=float64, chunksize=(1, 512, 5000), chunktype=numpy.ndarray>
 Coordinates:
   * y            (y) float64 4.1e+06 4.1e+06 4.1e+06 ... 4.084e+06 4.084e+06
   * x            (x) float64 4.598e+05 4.598e+05 ... 5.097e+05 5.098e+05
   * band         (band) int32 1
     spatial_ref  int32 0
 Attributes:
     scale_factor:  1.0
     add_offset:    0.0
     grid_mapping:  spatial_ref,
 'B08': <xarray.DataArray (band: 1, y: 1575, x: 5000)>
 dask.array<getitem, shape=(1, 1575, 5000), dtype=float64, chunksize=(1, 512, 5000), chunktype=numpy.ndarray>
 Coordinates:
   * y            (y) float64 4.1e+06 4.1e+06 4.1e+06 ... 4.084e+06 4.084e+06
   * x            (x) float64 4.598e+05 4.598e+05 ... 5.097e+05 5.098e+05
   * band         (band) int32 1
     spatial_ref  int32 0
 Attributes:
     scale_factor:  1.0
     add_o

In [2]:
from dask.distributed import Client, LocalCluster
import multiprocessing as mp
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
band_list=['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A',
           'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'info', 'metadata',
           'visual', 'overview', 'thumbnail']

def download_subset_image(download_status=False,stac_result=None,item_id_list=[],
                        item_list=[],aoi=None,target_epsg='',
                        band_list=band_list[:-5],
                        download_path='./sentinel_cog',name_suffix='',auto_folder=True):
    result_list=[]
    if item_list:
        with LocalCluster(n_workers=int(0.5 * mp.cpu_count()),
                          processes=False,
                          threads_per_worker=1,
                          memory_limit='2GB',
                          #ip='tcp://localhost:9895',
                          ) as cluster, Client(cluster) as client:
            for item in item_list:
                bands_dict={}
                for band in band_list:
                    img_name=item.properties['sentinel:product_id']
                    bands_dict['image_name']=img_name
                    band_url=item.assets[band]['href']
                    rds = rioxarray.open_rasterio(band_url, masked=True, chunks=(512,512))
                    #aoi data from http://geojson.io 
                    # get aoi as geopandas df
                    datajson=json.dumps(aoi)
                    target_area=gpd.read_file(datajson)
                    #https://geopandas.org/projections.html
                    target_area=target_area.to_crs(rds.rio.crs.to_string())
                    clipped =rds.rio.clip(target_area.geometry)
                     
                    if target_epsg:
                        # target_epsg='epsg:4326'
                        clipped = clipped.rio.reproject(target_epsg)
    
                    if download_status:
                        img_path=download_path+'/'+img_name
                        if not os.path.isdir(img_path):
                            os.mkdir(img_path)
                        img_name=band+'.tif'
                        clipped.rio.to_raster(img_path+'/'+img_name)
                       
                    bands_dict[band]=clipped.copy()
                    rds=None
                result_list.append(bands_dict)
                
        return result_list

In [3]:
%%time
subset_dask=download_subset_image(download_status=False,item_list=items_list,aoi=data,target_epsg='',
                        band_list=['B02','B08'],download_path='./sentinel_cog',name_suffix='',auto_folder=True)

NameError: name 'items_list' is not defined

In [140]:
import dask_try

In [145]:
%%time
ndviwitharray=dask_try.calculate_ndvi(subset_dask[0]['B02'],subset_dask[0]['B08'])

Wall time: 2.77 s


In [146]:
type(ndviwitharray)

numpy.ndarray

In [143]:
%%time
ndvidirecty=dask_try.direct_ndvi(subset_dask[0]['B02'],subset_dask[0]['B08'])

Wall time: 1.33 s


In [ ]:
from dask import 

In [136]:
red_xarray=None
nir_xarray=None
red=None
nir=None
red=None
nir=None
ndvi = None

In [ ]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    red_xarray=subset_dask[0]['B02']
    nir_xarray=subset_dask[0]['B08']
    red=red_xarray.persist()
    nir=nir_xarray.persist()
    red=red.values
    nir=nir.values
    ndvi = (nir.astype(float) - red.astype(float))/(nir + red)
    
    #2min 16s

In [137]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=False,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    red_xarray=subset_dask[0]['B02']
    nir_xarray=subset_dask[0]['B08']
    red=red_xarray.persist()
    nir=nir_xarray.persist()
    red=red.values
    nir=nir.values
    ndvi = (nir.astype(float) - red.astype(float))/(nir + red)
#2.31 s
    

Wall time: 2.32 s


In [75]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    red_xarray=subset_dask[0]['B02']
    nir_xarray=subset_dask[0]['B08']
    ndvi = (nir_xarray.astype(float) - red_xarray.astype(float))/(nir_xarray + red_xarray)
    ndvi.rio.to_raster('sen_cog_ndvi.tif')
    
    

Wall time: 2min 8s


In [89]:
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    ndvi_reproject = ndvi.rio.reproject_match(red_xarray)

In [90]:
ndvi_reproject.rio.crs

CRS.from_epsg(32637)

In [80]:
ndvi_4326 = ndvi.rio.reproject('epsg:4326')

In [113]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    con_utm=ndvi_4326.rio.reproject(dst_crs=ndvi.rio.crs,shape=ndvi.rio.shape)
    con_utm=con_utm.rio.reproject_match(ndvi)
    con_utm.rio.to_raster('ndvilatToUTM.tif')

Wall time: 3.05 s


In [124]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=False,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    con_utm=ndvi_4326.rio.reproject(dst_crs=ndvi.rio.crs,shape=ndvi.rio.shape)
    con_utm=con_utm.rio.reproject_match(ndvi)
    con_utm.rio.to_raster('ndvilatToUTM2.tif')

Wall time: 2.28 s


In [114]:
ndvi_4326.rio.reproject?

Signature: ndvi_4326.rio.reproject(dst_crs, resolution=None, shape=None, transform=None, resampling=<Resampling.nearest: 0>)
Docstring:
Reproject :obj:`xarray.DataArray` objects

Powered by `rasterio.warp.reproject`

.. note:: Only 2D/3D arrays with dimensions 'x'/'y' are currently supported.
    Requires either a grid mapping variable with 'spatial_ref' or
    a 'crs' attribute to be set containing a valid CRS.
    If using a WKT (e.g. from spatiareference.org), make sure it is an OGC WKT.

.. versionadded:: 0.0.27 shape
.. versionadded:: 0.0.28 transform

Parameters
----------
dst_crs: str
    OGC WKT string or Proj.4 string.
resolution: float or tuple(float, float), optional
    Size of a destination pixel in destination projection units
    (e.g. degrees or metres).
shape: tuple(int, int), optional
    Shape of the destination in pixels (dst_height, dst_width). Cannot be used
    together with resolution.
transform: optional
    The destination transform.
resampling: Resampling met

In [112]:
ndvi.rio.reproject_match?

Signature: ndvi.rio.reproject_match(match_data_array, resampling=<Resampling.nearest: 0>)
Docstring:
Reproject a DataArray object to match the resolution, projection,
and region of another DataArray.

Powered by `rasterio.warp.reproject`

.. note:: Only 2D/3D arrays with dimensions 'x'/'y' are currently supported.
    Requires either a grid mapping variable with 'spatial_ref' or
    a 'crs' attribute to be set containing a valid CRS.
    If using a WKT (e.g. from spatiareference.org), make sure it is an OGC WKT.

Parameters
----------
match_data_array:  :obj:`xarray.DataArray` | :obj:`xarray.Dataset`
    DataArray of the target resolution and projection.
resampling: Resampling method, optional
    See rasterio.warp.reproject for more details.


Returns
--------
:obj:`xarray.DataArray`:
    Contains the data from the src_data_array, reprojected to match
    match_data_array.
File:      c:\users\akif\.conda\envs\geo\lib\site-packages\rioxarray\rioxarray.py
Type:      method


In [83]:
def to_raster(in_xds, template_xds, out_file):
    in_xds = in_xds.rio.write_crs(template_xds.rio.crs)
    if template_xds.rio.nodata is not None:
        in_xds.attrs["_FillValue"] = template_xds.rio.nodata
    in_xds.rio.to_raster(out_file)



In [92]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    to_raster(ndvi_4326,red_xarray, "ndvifromred.tif")

Wall time: 2.25 s


In [121]:
geo=new_target.__geo_interface__

In [115]:
from rasterstats import zonal_stats, point_query

In [123]:
stats = zonal_stats(geo,ndvi,stats=['min', 'max', 'median', 'majority', 'sum'])

TypeError: invalid path or file: <xarray.DataArray (band: 1, y: 1575, x: 5000)>
dask.array<truediv, shape=(1, 1575, 5000), dtype=float64, chunksize=(1, 512, 5000), chunktype=numpy.ndarray>
Coordinates:
  * y            (y) float64 4.1e+06 4.1e+06 4.1e+06 ... 4.084e+06 4.084e+06
  * x            (x) float64 4.598e+05 4.598e+05 ... 5.097e+05 5.098e+05
  * band         (band) int32 1
    spatial_ref  int32 0

In [120]:
new_target=target_area.to_crs(ndvi.rio.crs.to_string())

In [155]:
aoi_gpd=target_area.to_crs(ndvi.rio.crs.to_string())

In [156]:
aoi_gpd.geometry

0    POLYGON ((430010.007 4148390.337, 457208.100 4...
Name: geometry, dtype: geometry

In [111]:
clipped =rds.rio.clip(aoi_gpd.geometry)

In [112]:
clipped.rio.to_raster('denemeee.tif')

In [157]:
aoi_gpd.to_file('utm',driver='ESRI Shapefile')

In [158]:
target_area.to_file('latlon',driver='ESRI Shapefile')

In [182]:
res={'type': 'Polygon',
 'coordinates': [[[37.876381589019886, 36.95257399124932],
   [37.861482438916816, 37.942072015005024],
   [38.5700476360887, 37.9467949936599],
   [38.48396578520234, 37.68856019631257],
   [38.238959267471955, 36.955451379212235],
   [37.876381589019886, 36.95257399124932]]]}

In [183]:
datajson=json.dumps(res)
target_area=gpd.read_file(datajson)
target_area.geometry

0    POLYGON ((37.87638 36.95257, 37.86148 37.94207...
Name: geometry, dtype: geometry

In [184]:
target_area.to_file('latlon2',driver='ESRI Shapefile')

In [185]:
aoi_gpd=target_area.to_crs(sdb37.rio.crs.to_string())

In [186]:
aoi_gpd.to_file('utm',driver='ESRI Shapefile')